# Segmenting and Clustering Neighborhoods in Toronto

## Getting Toronto neighborhoods

In [1]:
!conda install -yc conda-forge geocoder lxml cssselect
import pandas as pd
import numpy as np
import geocoder 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - cssselect
    - geocoder
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    click-7.0                  |             py_0      

*I use pandas to get data frames from a we page. Then I remove entries that have "Not assigned" in borough name and then replace ones in neighborhood names with borough name*

In [47]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = pd.DataFrame(df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)).reset_index()
df = df[df['Borough'] != 'Not assigned']
df[df['Neighborhood'] == 'Not assigned'] = df['Borough']
df = df.reset_index(drop = True)

df

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


*I use Bing API to get geocodindg of neigborhoods by their names.*

In [3]:
import time
BING_KEY = 'Aq7bYrs0DLI7HUs7AbCd0Ejxsr72-c_RVKm0t1p6ivr0Dne8fbEdie88uM9rQlvS'
def get_location(zip):
    #time.sleep(1)
    geo = geocoder.bing("{}, Toronto, Canada".format(zip), key = BING_KEY)
    #print(geo.json)
    return pd.Series(geo.latlng, index = ('Latitude', 'Longitude'))


In [45]:
neighborhoods = df.join(df['Postcode'].apply(lambda n: get_location(n)))

In [46]:
neighborhoods

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.810154,-79.194603
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784672,-79.158958
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.766289,-79.172890
3,M1G,Scarborough,Woburn,43.768288,-79.214111
4,M1H,Scarborough,Cedarbrae,43.769180,-79.238770
...,...,...,...,...,...
98,M9N,York,Weston,43.705570,-79.518616
99,M9P,Etobicoke,Westmount,43.696484,-79.529266
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.684505,-79.557915
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.742935,-79.584831


## Explore neighborhoods

In [6]:
import folium # map rendering library
import requests

In [7]:
address = 'Toronto, ON'

location = geocoder.bing(address, key = BING_KEY)
latitude, longitude = location.latlng
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.64868927001953, -79.38543701171875.


In [48]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'IHA2RZ534UBEJJ0OX0X3HCPDS4HQL4TQTCUHCYPEW3XLQDO3' # your Foursquare ID
CLIENT_SECRET = 'LASEJ4FGP0SZSRYNE4I3YE2MCWI2YWAIAAPSUX54EPELLAFD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IHA2RZ534UBEJJ0OX0X3HCPDS4HQL4TQTCUHCYPEW3XLQDO3
CLIENT_SECRET:LASEJ4FGP0SZSRYNE4I3YE2MCWI2YWAIAAPSUX54EPELLAFD


In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('{}... '.format(name), end = '')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        print("found {} venues.".format(len(results)))

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### As there is a limitation of API calls to Foursquare I just saved it to CSV and use it offline. In order to get data from Foursquare uncomment this cell

In [49]:
#tmp = neighborhoods.sort_values(['Neighborhood', 'Borough'])
#toronto_venues = getNearbyVenues(names=tmp['Neighborhood'],
#                                   latitudes=tmp['Latitude'],
#                                   longitudes=tmp['Longitude'],
#                                   limit = 100
#                                  )
#toronto_venues.to_csv('toronto-venues.csv', index = False)

Adelaide,King,Richmond... found 100 venues.
Agincourt... found 19 venues.
Agincourt North,L'Amoreaux East,Milliken,Steeles East... found 4 venues.
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown... found 12 venues.
Alderwood,Long Branch... found 8 venues.
Bathurst Manor,Downsview North,Wilson Heights... found 3 venues.
Bayview Village... found 3 venues.
Bedford Park,Lawrence Manor East... found 21 venues.
Berczy Park... found 16 venues.
Birch Cliff,Cliffside West... found 7 venues.
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe... found 5 venues.
Brockton,Exhibition Place,Parkdale Village... found 68 venues.
Business Reply Mail Processing Centre 969 Eastern... found 8 venues.
CFB Toronto,Downsview East... found 4 venues.
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara... found 71 venues.
Cabbagetown,St. James Town... found 44 venues.
Caledonia-Fairbanks... found 6 ven

In [50]:
toronto_venues = pd.read_csv('toronto-venues.csv')
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide,King,Richmond",43.650410,-79.382881,The Keg Steakhouse & Bar,43.649937,-79.384196,Steakhouse
1,"Adelaide,King,Richmond",43.650410,-79.382881,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,"Adelaide,King,Richmond",43.650410,-79.382881,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
3,"Adelaide,King,Richmond",43.650410,-79.382881,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
4,"Adelaide,King,Richmond",43.650410,-79.382881,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
...,...,...,...,...,...,...,...
2189,Woodbine Heights,43.689819,-79.310364,Bark 'N' Scratch,43.687388,-79.305479,Pet Store
2190,York Mills West,43.749180,-79.399178,TD Canada Trust Branch and ATM,43.750923,-79.401004,Bank
2191,York Mills West,43.749180,-79.399178,Tournament Park,43.751257,-79.399717,Park
2192,York Mills West,43.749180,-79.399178,Kitchen Food Fair,43.751298,-79.401393,Convenience Store


In [52]:
# Some venues have "Neighborhood" category and refer to "Downtown Toronto". I'll just remove them
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood']

In [53]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",99,99,99,99,99,99
Agincourt,19,19,19,19,19,19
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",4,4,4,4,4,4
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",12,12,12,12,12,12
"Alderwood,Long Branch",8,8,8,8,8,8
...,...,...,...,...,...,...
Willowdale West,10,10,10,10,10,10
Woburn,2,2,2,2,2,2
"Woodbine Gardens,Parkview Hill",14,14,14,14,14,14


In [54]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 262 uniques categories.


In [55]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.shape

(2192, 263)

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.020202,0.010101,0.0,0.000000,0.030303,0.000000,0.0,...,0.0,0.010101,0.0,0.000000,0.000000,0.010101,0.0,0.0,0.010101,0.0
1,Agincourt,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.052632,0.000000,0.0,0.0,0.000000,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.083333,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
96,Woburn,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
97,"Woodbine Gardens,Parkview Hill",0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.071429,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
98,Woodbine Heights,0.0,0.0,0.000000,0.000000,0.0,0.083333,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


#### First, let's write a function to sort the venues in descending order

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Cluster neighborhoods

In [20]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [78]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print("Got {} labels for {} neighborhoods".format(len(kmeans.labels_), len(toronto_grouped)))

Got 100 labels for 100 neighborhoods


In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Restaurant,Hotel,Bar,Asian Restaurant,Burger Joint,Gastropub,Breakfast Spot
1,Agincourt,Chinese Restaurant,Coffee Shop,Pizza Place,Badminton Court,Shanghai Restaurant,Bubble Tea Shop,Sandwich Place,Supermarket,Sushi Restaurant,Skating Rink
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Park,Gym,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Yoga Studio
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Park,Caribbean Restaurant,Fried Chicken Joint,Beer Store,Sandwich Place,Fast Food Restaurant,Hardware Store,Video Store
4,"Alderwood,Long Branch",Pizza Place,Gas Station,Coffee Shop,Pub,Gym,Pharmacy,Convenience Store,Sandwich Place,Farm,Falafel Restaurant


In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# and then we remove neighborhoods that have no venues at all (cluster label and most common venues are NaN)
toronto_merged = toronto_merged.dropna()

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, we examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we then assign a name to each cluster.

#### Cluster 1

In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]["1st Most Common Venue"].unique()

array(['Gym / Fitness Center', 'Park', 'Pharmacy', 'Convenience Store',
       'Food & Drink Shop', 'Pizza Place', 'Playground',
       'Accessories Store', 'Brewery', 'Coffee Shop'], dtype=object)

#### Cluster 2

In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]["1st Most Common Venue"].unique()

array(['Construction & Landscaping'], dtype=object)

#### Cluster 3

In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]["1st Most Common Venue"].unique()

array(['Fast Food Restaurant', 'Bar', 'Business Service', 'Train Station',
       'Convenience Store', 'Soccer Field', 'Gift Shop',
       'Chinese Restaurant', 'Pizza Place', 'Clothing Store',
       'Middle Eastern Restaurant', 'Café', 'Asian Restaurant',
       'Massage Studio', 'Home Service', 'Pet Store',
       'Sporting Goods Shop', 'Trail', 'Grocery Store', 'Coffee Shop',
       'Swim School', 'Gym', 'Theme Park', 'Sandwich Place', 'Garden',
       'Italian Restaurant', 'Mediterranean Restaurant', 'Playground',
       'Mexican Restaurant', 'Park', 'Eastern European Restaurant',
       'American Restaurant', 'Doner Restaurant', 'Burrito Place',
       'Shopping Mall'], dtype=object)

**Examination of the clusters shows there are 3 neighborhood clusters that can be distinguished:**
 - Cluster 1 - Can be called *residential* one. 
 - Cluster 2 - *Construction & landscaping*
 - Cluster 3 - Can be called *business and service* one